# Dancing particles

This project will be concerned with developing a simple code for the classical dynamics of interacting particles. Notice that there are already a number of Julia packages (such as [Molly](https://github.com/JuliaMolSim/Molly.jl) or [NBodySimulator](https://github.com/SciML/NBodySimulator.jl), which are able to do this in various contexts, so in practice one should rather resort to these packages instead of starting from scratch. Still, why not just get our hands dirty with what we know so far ;).

## Propagating a particle in time

Assume we are given a potential $V(x)$ for a single
particle with unit mass at position $x$. We want to propagate the particle in time.
That's integrating the famous Newton's equation of motion
$$ -\frac{dV}{dx} = \frac{d^2x}{dt^2}$$
in time. Defining the force map
$$ F_V = -\frac{dV}{dx}$$
this may be written equivalently:
$$\left\{\begin{array}{l}
\frac{dv}{dt} = F_V \\
\frac{dx}{dt} = v
\end{array}\right. $$

At first we will just apply forward-Euler: We discrete in time using the interval
$\Delta t$, which leads to sequences $t^{(n)} = n \Delta t + t^{(0)}$ and similarly $v^{(n)}$, $x^{(n)}$, etc:
$$\left\{\begin{array}{l}
v^{(n+1)} = v^{(n)} + F_V(x^{(n)}) \Delta t\\
x^{(n+1)} = x^{(n)} + v^{(n)} \Delta t\\
\end{array}\right. .$$
This is transformed to a Julia function in a single line:

In [ ]:
euler(F, Δt, xn, vn) = xn + vn * Δt, vn + F(xn) * Δt

where `F` is a function object to be passed to `euler`, for example:

In [ ]:
F(x) = -x
euler(F, 0.1, 0, 1)

Of course we can repeat this now and get a particle moving in phase space:

In [ ]:
Δt  = 0.1
x, v = 0, 1
for n = 1:10
    @show x, v
    x, v = euler(F, Δt, x, v)
end 

## Taking derivatives without headaches

First we consider the Harmonic potential. We plot the function and its derivative

In [ ]:
using Plots

In [ ]:
Vho(r, a=0.5)  = (r - a)^2   # Shifted harmonic oscillator
dVho(r, a=0.5) = 2r -2a      # Derivative

r = collect(-5:0.1:5)        # Create series of x-values
p = plot(r, Vho.(r), label="Vho a=0.5")
plot!(p, r, dVho.(r), label="∂Vho")
xaxis!(p, "relative radial distance")

Now, taking the derivative for the harmonic oscillator this is kind of ok, but for more complicated potentials $V$ this quickly becomes error prone and most importantly quite *boring*. The solution is **algorithmic differentiation**, where the Julia compiler is taught how to automatically trace the code and not only produce the function value, but also produce a derivative.

Without going into further details here, let us use the `Zygote` package to take the derivative on `Vho` as defined above:

In [ ]:
using Zygote

r = collect(-5:0.1:5)
p = plot(r, Vho.(r), label="Vho a=0.5")
xaxis!(p, "relative radial distance")
plot!(p, r, Vho'.(r), label="∂Vho")   # Notice the dash

With ease we add the second derivative to the plot:

In [ ]:
plot!(p, Vho'', label="∂∂Vho")

An important thing to note is that this **really generates the derivative code** instead of numerical differentiation (i.e. finite-differences):

In [ ]:
@code_llvm debuginfo=:none Vho'(1.0)

Now let us consider more complicated potentials, for example the **Morse potential**, which is a common model for a chemical bond:
$$ V_\text{Morse} = D (1 - \exp(-\alpha (r - r_0)))^2 - D$$

In [ ]:
Vmorse(r; r₀=0.7, α=1.3, D=10) = D*(1 - exp(-α * (r - r₀)))^2 - D

In [ ]:
r = collect(0:0.1:7)
p = plot(r, Vmorse.(r), label="Vmorse", ylim=(-25, 25))
xaxis!(p, "Radial distance")
plot!(p, r, Vmorse'.(r), label="Vmorse'")
plot!(p, r, Vmorse''.(r), label="Vmorse''")

This works for higher dimensions and more complicated expressions, too, we will use this in a sec.

## Inspecting Euler dynamics in 1D

Now we actually want to see things! We define a plot function to plot the potential and animate the particle over time:

In [ ]:
using Printf

# The arguments after the ; are so-called keyword arguments, they can be omitted
# and then the default value after the = is used
function plot_dynamics_line(V, Δt, n_steps; x0=0.0, v0=randn(), integrator=euler,
                            xlim=(-5, 5), ylim=(0, 10))
    t, x, v = 0.0, x0, v0  # Initialise state
    
    # Compute potential values (for plotting)
    xrange = xlim[1]:0.1:xlim[2]
    Vrange = V.(xrange)
    
    # @gif is a macro to "record" an animation of the dynamics,
    # each loop iteration gives a frame
    @gif for i in 1:n_steps
        # Propagate dynamics (notice the derivative)
        x, v = integrator(x -> -V'(x), Δt, x, v)
        t += Δt

        # Plot potential
        p = plot(xrange, Vrange, label="Potential", xlim=xlim, ylim=ylim)
        
        # Plot the particle and its velocity (as an arrow)
        timestr = @sprintf "%.2f" t   # Format time as a nice string
        scatter!(p, [x], [V(x)], label="Particle at t=$timestr")
        plot!(p, [(x, V(x)), (x + 0.5v, V(x))], arrow=1.0,
              label="particle velocity / 2")
    end
end

In [ ]:
# Now let's actually see it ....
plot_dynamics_line(Vho, 0.1, 200)

Wow ... that's strange. 
Our plot points at the well-known problem that energy is not conserved for a simple forward Euler scheme.
We can also diagnose this using a phase-space plot, where the time evolution of particle position $x$ and particle velocity $v$ is shown. This should be a closed curve if energy is conserved, so let's see ...

In [ ]:
function plot_phase_space(A, Δt, n_steps; x0=0.0, v0=randn(), integrator=euler,
                          xlim=(-7, 7), ylim=(-7, 7))
    x, v = x0, v0
    p = plot([x], [v], xlim=xlim, ylim=ylim, label="", xaxis="position", yaxis="velocity")
    @gif for N in 1:n_steps
        x, v = integrator(A, Δt, x, v)
        push!(p, x, v)  # Add new positions to the plot ...
    end
end

In [ ]:
plot_phase_space(x -> -Vho'(x), 0.1, 200)

**Exercise:**

A much more stable integrator than the `euler` we used so far is the verlocity Verlet:

$$\left\{\begin{array}{l}
x^{(n+1)} = x^{(n)} + v^{(n)} \Delta t + \frac{F_V(x^{(n)})}{2} \Delta t^2\\
v^{(n+1)} = v^{(n)} + \frac{F_V(x^{(n))} + F_V(x^{(n+1)})}{2} \Delta t\\
\end{array}\right. $$

- Program this algorithm, taking care that it supports multi-dimensional positions and velocities as well. In practice we would like to avoid recomputing $F_V(x)$ as much as possible, since this is usually the expensive step of the dynamics. For our purposes there is no need to keep an eye on that for now.
- How does the previous dynamics look like in this example. Does this algorithm conserve energy (phase-space plot)?
- Also look at the Morse potential

In [ ]:
# solution
# verlet(F, Δt, xn, vn) =

In [ ]:
# Some example plots and parameters:
# plot_dynamics_line(Vho, 0.1, 200, integrator=verlet, ylim=(0, 2.5), xlim=(-1, 3))
# plot_phase_space(x -> -Vho'(x), 0.1, 200, integrator=verlet, xlim=(-2, 2), ylim=(-2, 2))
# plot_dynamics_line(Vmorse, 0.03, 200, integrator=verlet, xlim=(-1, 4), ylim=(-10, 5), x0=2.0)

# Multiple particles

Now we want to simulate multiple identical partices in 2D. For a system of $N$ particles in 2D, we define the particle positions as the matrix
$$ \textbf{x} = \left(
    \begin{array}{cccc}
        x_{1x} & x_{2x} & \cdots & x_{Nx} \\
        x_{1y} & x_{2y} & \cdots & x_{Ny}
    \end{array}
   \right) = \left( \vec{x}_1 \vec{x}_2 \cdots \vec{x}_N \right). $$
with the individual particle vectors as columns.
We assume our particles interact via the idential pair potential $V_\text{pair}(r)$
depending only on particle distance $r$. The total potential is therefore
$$ V_\text{tot}(\textbf{x}) = \sum_{i=1}^N \sum_{j=i+1}^N V_\text{pair}(\| \vec{x}_i - \vec{x}_j \|). $$

### Exercise
Program the total potential function for a matrix $\textbf{x}$. A useful function is `norm` from the `LinearAlgebra` package.

In [ ]:
using LinearAlgebra

function Vtot(Vpair, x)
    # You're code here ...
end

Now we want to plot the dynamics in a plane. In the following function the acceleration is computed using automatically generated derivatives.

In [ ]:
function plot_dynamics_plane(Vpair, Δt, n_steps; x0=randn(2, 2), v0=zero(x0), integrator=verlet,
                             xlim=(-3, 3), ylim=(-3, 3))
    # Total acceleration via automatic differentiation
    V(x) = Vtot(Vpair, x)
    Atot(x) = -V'(x)
    
    t, x, v = 0, x0, v0  # Initialise state
    @gif for i in 1:n_steps
        # Propagate dynamics
        x, v = integrator(Atot, Δt, x, v)
        t += Δt
        timestr = @sprintf "%.2f" t   # Format time
        
        # Plot the particles and their velocities
        p = scatter(x[1, :], x[2, :], xlim=xlim, ylim=ylim, label="Particles at t=$timestr")
        label = "Velocity / 4"
        for (xi, vi) in zip(eachcol(x), eachcol(v))
            plot!(p, [Tuple(xi), Tuple(xi + 0.25vi)], arrow=1.0, colour=:red, label=label)
            label = ""
        end
    end
end

In [ ]:
Δt = 0.07
n_steps = 300
x0 = [[0.; 0.] [1.; 0.] [-1.; 0.0]]
plot_dynamics_plane(Vmorse, Δt, n_steps; x0=x0)

## A few nice examples

In [ ]:
Δt = 0.07
n_steps = 300
x0 = [[0.; 0.] [1.; 0.] [-1.; 0.15]]
plot_dynamics_plane(Vmorse, Δt, n_steps; x0=x0)

In [ ]:
Δt = 0.05
n_steps = 300
x0 = [[0.; 1.] [1.; 0.] [-1.; 0] [0; -1.2]]
plot_dynamics_plane(Vmorse, Δt, n_steps; x0=x0, ylim=(-10, 10))

In [ ]:
Δt = 0.05
n_steps = 300
x0 = 4randn(2, 10)
plot_dynamics_plane(Vmorse, Δt, n_steps; x0=x0, xlim=(-10, 10), ylim=(-10, 10))

## Open-ended exercise

There are many ways one could go from here. Some suggestions (roughly ordered by difficulty):

#### 1. Performance improvements
Take a look at [03_Performance_Engineering.ipynb](03_Performance_Engineering.ipynb) and afterwards try to get your evaluation of `Vtot` as fast as possible. For example, try to reduce the timings of:

In [ ]:
using BenchmarkTools
x = randn(2, 5000)
@btime Vtot($Vmorse, $x)

#### 2. Parallelism and speed:
   * The expensive step in the simulation is the double loop over particles needed to compute Vtot.
   * Some suggestions were discussed in [../08_Multithreading_Basics.ipynb](../08_Multithreading_Basics.ipynb)
   
   
   
#### 3. GPU support:
   * Since all operations we need are just linear algebra, just replacing standard arrays by `CuArray`s
     should be everything what is needed (see [../07_Linear_Algebra_Profiling.ipynb](../07_Linear_Algebra_Profiling.ipynb)
   * *Disclaimer:* I did not test this.